> 07 MIssile Command  
- 낙하하는 미사일을 요격하는 게임입니다. 마우스 커서로 조준해서 클릭으로 요격 미사일을 발사하세요. 포풍의 소용돌이에 휘말리면 적 미사일도 폭발합니다. 화면 아랫 부분에 있는 집이 전멸하면 게임오버입니다

In [1]:
## missile_command
import sys
import pygame
from random import randint
from math import hypot  #두점사이의 거리를 구하는 함수
from pygame.locals import QUIT, MOUSEBUTTONDOWN,Rect,MOUSEMOTION

class House:
    def __init__(self, xpos):
        self.rect=Rect(xpos,550,40,40)
        self.exploded=False
        strip =pygame.image.load("strip.png")
        self.images=(pygame.Surface((40,40),pygame.SRCALPHA),
                     pygame.Surface((40,40),pygame.SRCALPHA))
        self.images[0].blit(strip,(0,0),(0,0,20,20))
        self.images[1].blit(strip,(0,0),(20,0,20,20))
    
    def draw(self):
        if self.exploded:
            SURFACE.blit(self.images[1],self.rect)
        else:
            SURFACE.blit(self.images[0],self.rect)

class Missile: #낙하하는 미사일 객체
    def __init__(self):
        self.max_count =500 #발사부터 지면에 떨어지는 필요한 시간 카운트
        self.interval = 1000 #미사일 발사간격
        self.pos =[0,0] #미사일 현재위치 낙하좌표 낙하시작점 도착점
        self.cpos = [0,0] #미사일 현재위치
        self.firetime=0 #미사일 발사시작시간
        self.radius= 0 #폭발반경
        self.reload(0)  #미사일을 재사용해야하므로 실제적인 초기화를 한다.

    def reload(self,time_count):
        #미사일재초기화(낙하 후 나 폭발후)
        house_x= randint(0,12)*60 + 20
        self.pos=(randint(0,800),house_x)
        self.interval= int(self.interval* 0.9)
        self.firetime = randint(0,self.interval) + time_count
        self.radius=0
        self.cpos=[0,0]

    def tick(self,time_count,shoot,houses):

        #미사일 1프레임마다 처리
        is_hit= False
        elapsed = time_count - self.firetime
        if elapsed <0:  #미사일발사시각전이면 아무것도 하지 않고 되돌아갑니다.
            return

        if self.radius > 0: #폭발중이면
            self.radius += 1
            if self.radius > 100:
                self.reload(time_count)
        else:
            self.cpos[0] = (self.pos[1]-self.pos[0]) * elapsed / self.max_count + self.pos[0]
            self.cpos[1] = 550 * elapsed / self.max_count
            #요격빔에 맞아 떨어졌나 확인
            diff = hypot(shoot.shot_pos[0] - self.cpos[0], shoot.shot_pos[1] - self.cpos[1])
            if diff < shoot.radius:
                is_hit = True
                self.radius = 1  #폭발시작
            
            #지면에 충돌했나 확인
            if elapsed >= self.max_count:
                self.radius = 1  #폭발시작
                #집에 충돌했나 확인
                for house in houses:
                    if hypot(self.cpos[0]- house.rect.center[0], self.cpos[1]- (house.rect.center[1])) < 30:
                        house.exploded = True
        return is_hit
    
    def draw(self): #미사일그리기
        pygame.draw.line(SURFACE,(0,255,255),(self.pos[0],0),self.cpos)
        if self.radius >0:  #폭발중이면
           rad = self.radius if self.radius < 50 else 100 - self.radius
           pos =(int(self.cpos[0]), int(self.cpos[1]))
           pygame.draw.circle(SURFACE,(0,255,255),pos,rad)

class Shoot: #스스로폭발하는 객체

    def __init__(self):
        self.scope= (400,300)
        self.shot_pos= (0,0)#조준기의 중심좌표
        self.image= pygame.image.load("scope.png")  #조준기이미지
        self.count=0    #발사하고나서의카운트
        self.fire=False  #발사했는가
        self.radius=0   #폭풍반경
        self.shot_pos=(0,0) #현재탄두의위치
    def tick(self):
        #폭발중인빔의 위치,상태를갱신
        if self.fire:
            self.count +=1

            if 100 <=self.count < 200:
                self.radius +=1
            elif 200 <= self.count < 300:
                self.radius -=1
            elif self.count >=300:
                self.fire =False
                self.count =0

    def draw(self):
        #빔그리기
        rect= self.image.get_rect()
        rect.center = self.scope
        SURFACE.blit(self.image,rect)
        if not self.fire:
            return
        #폭발중이면
        if self.radius == 0 and self.count <100 :
            ratio= self.count /100
            ypos = 600 - (600 - self.shot_pos[1]) * ratio
            x_left =int((self.shot_pos[0]) * ratio)
            x_right =int(800 - (800 - self.shot_pos[0]) * ratio)
            pygame.draw.line(SURFACE,(0,255,0),(0,600),(x_left,ypos))
            pygame.draw.line(SURFACE,(0,255,0),(800,600),(x_right,ypos))
        elif self.radius >0:
            pygame.draw.circle(SURFACE,(0,255,0),self.shot_pos,self.radius)
#전역변수
pygame.init()
SURFACE = pygame.display.set_mode((800, 600))
FPSCLOCK = pygame.time.Clock()


def main():
    game_over= False
    missiles= []
    score = 0
    time_count = 0
    shoot = Shoot()
    houses =[]

    scorefont = pygame.font.SysFont(None,36)
    sysfont = pygame.font.SysFont(None,72)
    message_over= sysfont.render("GAME OVER!!",True,(0,255,255))
    message_rect = message_over.get_rect()
    message_rect.center = (400,300)

    for i in range(13):
        houses.append(House(i*60 +20))

    while len(missiles) < 8:
        missiles.append(Missile())
    while True:
        time_count +=1
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == MOUSEMOTION:
                shoot.scope = event.pos
            elif event.type == MOUSEBUTTONDOWN:
                if not shoot.fire:
                    shoot.shot_pos = shoot.scope
                    shoot.fire = True
                    
        #1프레임마다처리
        exploded = len(list(filter(lambda h: h.exploded, houses)))
        game_over = exploded == 13
        if not game_over:
            for missile in missiles:
                is_hit = missile.tick(time_count,shoot,houses)
                if is_hit:
                    score += 100
            shoot.tick()
        #그리기
        SURFACE.fill((0,0,0))
        shoot.draw()
        for house in houses:
            house.draw()
        for missile in missiles:
            missile.draw()
        score_str= str(score).zfill(6)
        score_image = scorefont.render(score_str,True,(255,255,255))
        SURFACE.blit(score_image,(700,10))

        if game_over:
            SURFACE.blit(message_over,message_rect)
        pygame.display.update()
        FPSCLOCK.tick(30)

if __name__ =='__main__':
    main()


c:\Users\고창섭\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\고창섭\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
